<a href="https://colab.research.google.com/github/wirarama/auditenergi/blob/master/ARMTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/file/d/1g2hR8VCpBmbQbEnYccyZLeANz490eXC2'
trash, id = link.split('/d/')
print (id)
downloaded = drive.CreateFile({'id':id})

1g2hR8VCpBmbQbEnYccyZLeANz490eXC2


In [0]:
downloaded.GetContentFile('vgsales.csv')

In [0]:
df = pd.read_csv('vgsales.csv')

In [0]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('sqlite:///:memory:')
df.to_sql('vgsales',engine)

In [0]:
df.columns

In [0]:
df['Platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [0]:
def ARMformat(a,b,tb):
    collect = []
    for x in df[a].unique():
        q = "SELECT "+b+" FROM "+tb+" WHERE "+a+"='"+x+"'"
        res = pd.read_sql_query(q,engine)
        res=res[b].unique()
        resclean = []
        #pre = res[0]
        for i in res:
            if i!=None or i!='null' or i!='NaN':
                resclean.append(i)
                #pre = i
            #else
                #resclean.append(pre)
        resmin = resclean[0:10] #terpaksa biar cepat saja
        collect.append(list(resmin))
    return collect

In [0]:
data = ARMformat('Platform','Publisher','vgsales')
len(data)

31

In [0]:
i = 0
pre = df['Platform'].unique()
for dt in data:
    print(pre[i],dt,"\n")
    i+=1

Wii ['Nintendo', 'Ubisoft', 'Sega', '505 Games', 'LucasArts', 'Activision', 'Take-Two Interactive', 'Electronic Arts', 'Warner Bros. Interactive Entertainment', 'Disney Interactive Studios'] 

NES ['Nintendo', 'Palcom', 'Enix Corporation', 'Namco Bandai Games', 'Capcom', 'Konami Digital Entertainment', 'Hudson Soft', 'SquareSoft', 'Ocean', 'BPS'] 

GB ['Nintendo', 'Eidos Interactive', 'Konami Digital Entertainment', 'Electronic Arts', 'Enix Corporation', 'Namco Bandai Games', 'Capcom', 'Activision', 'SquareSoft', 'Imagineer'] 

DS ['Nintendo', '505 Games', 'Sega', 'LucasArts', 'Electronic Arts', 'Activision', 'Disney Interactive Studios', 'Warner Bros. Interactive Entertainment', 'Ubisoft', 'Square Enix'] 

X360 ['Microsoft Game Studios', 'Take-Two Interactive', 'Activision', 'Bethesda Softworks', 'Electronic Arts', 'Ubisoft', 'Warner Bros. Interactive Entertainment', 'Capcom', 'Eidos Interactive', 'MTV Games'] 

PS3 ['Take-Two Interactive', 'Activision', 'Sony Computer Entertainment',

FPGrowth

In [0]:
!pip install pyfpgrowth

     |████████████████████████████████| 1.6MB 2.9MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5479 sha256=7583a09419f51e27df9ca1fb44e18a2abd1d63a85c7dbd3e5486a06b0a18c6e9
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [0]:
import pyfpgrowth

In [0]:
patterns = pyfpgrowth.find_frequent_patterns(data, 12)
for p,d in patterns.items():
    print(p,":",d)

('Konami Digital Entertainment',) : 12
('Namco Bandai Games',) : 13
('Sega',) : 14
('Electronic Arts',) : 16
('Activision', 'Electronic Arts') : 15
('Activision',) : 18


In [0]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.5)
for p,d in rules.items():
    print(p,"=>",d)

('Activision',) => (('Electronic Arts',), 0.8333333333333334)
('Electronic Arts',) => (('Activision',), 0.9375)


Apriori

In [0]:
!pip install efficient_apriori

In [0]:
from efficient_apriori import apriori

In [0]:
itemsets, rules = apriori(data, min_support=0.4,  min_confidence=0.2)

In [0]:
hasil = []
for x in rules:
    hasil.append([str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction])
    print(str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction,sep="\t")

('Electronic Arts',)=>('Activision',)	0.4838709677419355	0.9375	1.6145833333333335	6.709677312000001
('Activision',)=>('Electronic Arts',)	0.4838709677419355	0.8333333333333334	1.6145833333333335	2.903225789032259
